### Create a unique identifier for city column using SQL
In this exercise, I will perform the following steps:
- Establish a connection link with the SQL DB in MySQL workbench
- Move the cities dataframe (columns: "City" and "Elevation") to SQL DB uniquely identifying the city names (table column "city_id")
- Read the SQL table in python and merge city_id back into the webscraped dataframe
- Move data to the following SQL tables:
    1. Countries (with columns: "city_id", "Country", "country_2c")
    2. Populations (with columns: "city_id", "Population")
    3. Coordinates (with columns: "city_id", "latitude", "longitude")

#### Import packages

In [1]:
import pandas as pd
import sqlalchemy

# Get the api key
from keys import MySQL_bootcamp

#### Create the connection link

In [2]:
schema = "gans_cities"
host = "127.0.0.1"
user = "root"
password = MySQL_bootcamp
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Import the merged dataframe

In [3]:
df_cities_ws_cleaned = pd.read_csv("data/df_cities_ws_cleaned.csv")

#### Create a unique identifier for the city using SQL

In [4]:
df = df_cities_ws_cleaned.copy()
df_city = df[["City", "Elevation (in m)"]]
df_city=df_city.rename(columns= {"Elevation (in m)": "Elevation"})

df_city.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

20

#### Import SQL table and merge city_id in the original dataframe

In [5]:
df_city_sql = pd.read_sql("cities", con=connection_string)
df_sql_city_id = df.merge(df_city_sql, on = "City", how="left")
df_sql_city_id

,City,country_2c,latitude,longitude,is_capital,Country,Elevation (in m),Population,city_id,Elevation
0,Berlin,DE,52.5167,13.3833,True,Germany,34.0,3576873,1,34
1,Hamburg,DE,53.5500,10.0000,False,Germany,23.0,1945532,2,23
2,Munich,DE,48.1372,11.5755,False,Germany,520.0,1512491,3,520
3,Cologne,DE,50.9422,6.9578,False,Germany,37.0,1073096,4,37
4,Paris,FR,48.8566,2.3522,True,France,35.0,2102650,5,35
5,Nice,FR,43.7034,7.2663,False,France,10.0,348085,6,10
6,Rome,IT,41.8931,12.4828,True,Italy,21.0,2860009,7,21
7,Milan,IT,45.4669,9.1900,False,Italy,120.0,1371498,8,120
8,Warsaw,PL,52.2167,21.0333,True,Poland,100.0,1863056,9,100
9,Barcelona,ES,41.3825,2.1769,False,Spain,12.0,1620343,10,12


#### Create countries table in SQL

In [6]:
df_country = df_sql_city_id[["city_id", "country_2c", "Country"]]

df_country.to_sql('countries',
                  if_exists='append',
                  con=connection_string,
                  index=False)

20

#### Create a population table in SQL

In [7]:
df_pop = df_sql_city_id[["city_id", "Population"]]
df_pop = df_pop.copy()
df_pop["Year_retrieved"] = 2024

df_pop.to_sql('populations',
                  if_exists='append',
                  con=connection_string,
                  index=False)

20

#### Create a coordinates table

In [8]:
df_coord = df_sql_city_id[["city_id", "latitude", "longitude"]]
df_coord.to_sql('coordinates',
                  if_exists='append',
                  con=connection_string,
                  index=False)

20

#### Save dataframe as csv

In [9]:
df_sql_city_id.to_csv("data/df_sql_city_id.csv", sep=',', index=False, encoding='utf-8')